In [1]:
import pandas as pd
import numpy as np
import os, string, re, zipfile
import tensorflow as tf
import keras
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Perceptron

In [2]:
# Check if file exists
if not os.path.exists('..\\Data\\final_train.csv'):
    # Go back one directory
    os.chdir('..\\Data')

    # Download the dataset from Kaggle
    !kaggle datasets download -d jdragonxherrera/augmented-data-for-llm-detect-ai-generated-text

    zip_path = "augmented-data-for-llm-detect-ai-generated-text.zip"  # replace with the path to your zip file
    extract_path = "."  # replace with the path where you want to extract the files
    
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    
    os.chdir('..\\Notebooks')

In [3]:
# Augmented data for LLM - Detect AI Generated Text by jdragonxherrera on Kaggle
df_train = pd.read_csv('../Data/final_train.csv')
df_test = pd.read_csv('../Data/final_test.csv')
df = pd.concat([df_train, df_test])
df

,text,label
0,We should keep the Electoral College for a num...,0
1,More and more money is spent on building theat...,1
2,Limiting car usage can actually be effective b...,0
3,"Dear Mrs. Smith,\n\nI am writing to you today ...",1
4,"Dear Principal,\n\nAfter school or during scho...",0
...,...,...
86582,Dear Principal: I think we should have cell ph...,0
86583,Dear Teacher_NAME\n\nI think that if you try t...,0
86584,"Venus is sometimes called the ""meaning Star."" ...",0
86585,The Seagoing Cowboy Bros\n\nDo you like going ...,0


In [4]:
max_features = 20000
embedding_dim = 64
sequence_length = 500

In [5]:
shuffled_df = df.sample(frac=1, random_state=1818)
# Remove 80% of the data
num_samples_to_remove = int(0.8 * len(shuffled_df))
remaining_df = shuffled_df.iloc[num_samples_to_remove:]


train_df, remaining_df = train_test_split(remaining_df, train_size=0.8, random_state=1818)
val_df, test_df = train_test_split(remaining_df, train_size=0.5, random_state=1818)

def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, f"[{re.escape(string.punctuation)}]", ""
    )

vectorize_layer = keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

ds = tf.data.Dataset.from_tensor_slices((df['text'].values))
ds = ds.batch(512)
vectorize_layer.adapt(ds)

# Vectorize the data

train_text = np.array(vectorize_layer(train_df['text']))
val_text = np.array(vectorize_layer(val_df['text']))
test_text = np.array(vectorize_layer(test_df['text']))

train_labels = np.array(train_df['label'])
val_labels = np.array(val_df['label'])
test_labels = np.array(test_df['label'])

Initial model testing resulted in the following results, tested on data from the augmented-llm-data and on the data provided by the competition.

- Embedding, Conv1D, and Dense dim of 128
    - trained on 3 epochs had a test accuracy of 0.9905 and 0.9811 and took 2m 42.4s
    - trained on 6 epochs had a test accuracy of 0.9905 and 0.9601 and took 5m 17.6s
- Embedding, Conv1D, and Dense dim of 64
    - trained on 3 epochs had a test accuracy of 0.9983 and 0.9800 and took 1m 10.1s
    - trained on 6 epochs had a test accuracy of 0.9908 and 0.9586 and took 2m 19.3s

Extra epochs resulted in lower accuracy on the competition test set. For that reason, 3 epochs were chosen.

As the competition also has a 'Training Efficiency' portion, a low train time was preferred and a dim of 64 was chosen.

In [6]:
# A integer input for vocab indices.
inputs = keras.Input(shape=(None,), dtype="int64")

x = layers.Embedding(max_features, 64)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(64, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(64, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# Add a vanilla hidden layer:
x = layers.Dense(64, activation="relu")(x)
x = layers.Dropout(0.5)(x)

# Output layer
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model = keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

epochs = 3

# Fit the model using the train and val datasets.
model.fit(train_text, train_labels, validation_data=(val_text, val_labels), epochs=epochs)


Epoch 1/3

2168/2168 [==============================] - 24s 11ms/step - loss: 0.0868 - accuracy: 0.9658 - val_loss: 0.0281 - val_accuracy: 0.9922
Epoch 2/3
2168/2168 [==============================] - 23s 11ms/step - loss: 0.0194 - accuracy: 0.9939 - val_loss: 0.0185 - val_accuracy: 0.9941
Epoch 3/3
2168/2168 [==============================] - 22s 10ms/step - loss: 0.0108 - accuracy: 0.9968 - val_loss: 0.0175 - val_accuracy: 0.9942


In [7]:
# Take generated essays and student essays provided by the competition
combined_data = pd.read_csv('../Data/Detect AI Generated Text/combined_essays.csv')

# Drop the id, prompt_id, text_len, and model columns
combined_data = combined_data.drop(['id', 'prompt_id', 'text_len', 'model'], axis=1)

# Rename the 'generated' column to 'label'
combined_data = combined_data.rename(columns={'generated': 'label'})

# Convert the label column to a boolean and the text column to a string
combined_data['label'] = combined_data['label'].astype(bool)
combined_data['text'] = combined_data['text'].astype(str)

# Create a test set from the combined data for the keras model
cd_text = np.array(vectorize_layer(combined_data['text']))
cd_labels = np.array(combined_data['label'])

In [8]:
print("Evaluate on test data")
results = model.evaluate(test_text, test_labels)
print("test loss, test acc:", results)
results = model.evaluate(cd_text, cd_labels, batch_size=128)
print("test loss, test acc:", results)

Evaluate on test data
271/271 [==============================] - 0s 1ms/step - loss: 0.0267 - accuracy: 0.9933
test loss, test acc: [0.0267432052642107, 0.9933118224143982]
22/22 [==============================] - 0s 3ms/step - loss: 0.0694 - accuracy: 0.9800
test loss, test acc: [0.06940004229545593, 0.9800435304641724]


The competition allowed for the best of 3 notebooks to be used.

Initial:
- Embedding, Conv1D, and Dense dim of 64
    - trained on 20% of the augmented-llm-data had a test accuracy of 0.9933 and 0.9800 and took 1m 10.1s

Best:
- Embedding, Conv1D, and Dense dim of 64
    - trained on 60% of the augmented-llm-data had a test accuracy of 0.9976 and 0.9851 and took 3m 27s

In [9]:
# Vectorize the data for the Perceptron
vectorizer = TfidfVectorizer(max_features=max_features)
vectorizer.fit(df['text'])

# Vectorize the train and test data
train_vectors = vectorizer.transform(train_df['text'])
test_vectors = vectorizer.transform(test_df['text'])

# Create additonal test set from the combined data for the perceptron
cd_vectors = vectorizer.transform(combined_data['text'])
cd_labels = combined_data['label'].to_numpy()

In [97]:
perceptron_model = Perceptron(max_iter=1000)
perceptron_model.fit(train_vectors, train_labels)

# Make predictions on the test set
predictions = perceptron_model.predict(test_vectors)

# Evaluate the accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f"Accuracy: {accuracy:.2f}")

# classification report
print(classification_report(test_labels, predictions))

# confusion matrix
print(confusion_matrix(test_labels, predictions))

Accuracy: 0.99
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5508
           1       0.99      0.99      0.99      3164

    accuracy                           0.99      8672
   macro avg       0.99      0.99      0.99      8672
weighted avg       0.99      0.99      0.99      8672

[[5486   22]
 [  45 3119]]


In [110]:
class Classifier():
    def __init__(self, tensor_model, perceptron_model):
        self.tensor_model = tensor_model
        self.perceptron_model = perceptron_model

    def fit(self, X, y):
        # No training needed for TensorFlow model in this case
        pass

    def predict(self, X):
        # Convert the dataset to the expected format
        tensor_X = np.array(vectorize_layer(X))
        tensor_predictions = self.tensor_model.predict(tensor_X)
        tensor_predictions = np.array(tensor_predictions).flatten()
        perceptron_X = vectorizer.transform(X)
        perceptron_predictions = self.perceptron_model.predict(perceptron_X)
        # Turn the perceptron predictions into a 1D array of floats
        perceptron_predictions = np.array(perceptron_predictions).flatten().astype(float)
        # Average the predictions along the array axis
        predictions = np.mean([tensor_predictions, perceptron_predictions], axis=0)
        # Convert the predictions to binary
        predictions = np.where(predictions > 0.5, 1, 0)
        return predictions
    
classifier = Classifier(tensor_model=model, perceptron_model=perceptron_model)
train_df, test_df = train_test_split(shuffled_df, train_size=0.8, random_state=42)

train_labels2 = train_df['label'].to_numpy()
test_labels2 = test_df['label'].to_numpy()
# Get predictions from each model
predictions = classifier.predict(test_df['text'])  # Adjust according to your TensorFlow model

print(predictions[:5], test_labels2[:5])

# Evaluate the accuracy
accuracy = accuracy_score(test_labels2, predictions)
print(f"Accuracy: {accuracy:.2f}")

# Evaluate the accuracy
accuracy = accuracy_score(test_labels2, predictions)
print(f"Binary Accuracy: {accuracy:.2f}")

# classification report
print(classification_report(test_labels2, predictions))

# confusion matrix
print(confusion_matrix(test_labels2, predictions))

2710/2710 [==============================] - 4s 2ms/step
[1 1 1 0 0] [1 1 1 0 0]
Accuracy: 0.99
Binary Accuracy: 0.99
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     55334
           1       0.99      0.99      0.99     31379

    accuracy                           0.99     86713
   macro avg       0.99      0.99      0.99     86713
weighted avg       0.99      0.99      0.99     86713

[[55144   190]
 [  349 31030]]


Combining the Keras and Perceptron models into a classifier results in an accuracy of 99.378%.

My notebook tested on 46% of the competition's final test data achieved an accuracy of 85.67% and placed 3053 out of 4359.

Final results on 54% of test data were 74.25% and placed 2882 out of 4359.